# Shark Attacks

In [186]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re

### First look
- Vemos que hay información repetida como por ejemplo que el numero de caso y la fecha es lo mismo. pero si nos fijamos, antes del 2000 el número de caso  se establecia de otrra forma.

- La columna del nombre de la persona también es irrelevante en este caso en el que queremos obtener conclusiones genéricas. Con saber el sexo el suficiente.
- La columna del investigador tampoco es relevante si nuestro análisis se centra en estudiar los ataques. 

- Faltan más de la mitad de los datos sobre la hora, a demás son datos muy variados. Sin embargo, es facil observar que la mayoría se producen por la tarde.


In [187]:
data = pd.read_csv("../attacks.csv",encoding='ISO-8859-1')
data.head(6305)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,...,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,NaN,NaN
6301,ND.0001,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,...,NaN,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0001,ND.0001,2.0,NaN,NaN
6302,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6304.0,NaN,NaN
6303,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6305.0,NaN,NaN


**Una de las cosas que me parecen mas relevantes es que a partir de la fila 6302, todos los datos son NaN. Es decir, hay casi 20.000 datos en cada columna que no son un número. Vamos a deshacernos de esas filas.**

In [188]:
data = data[:6302]

También se podria haber hecho así, pero de esta forma no se incluían las filas de las 6302 a las 6310 porque aunque no están bien, no estaban duplicadas.
```python3
duplicateRows = data[data.duplicated()]
print(duplicateRows)
```

# Renombrar columnas 

In [189]:
data.rename(columns={'Sex ':'Sex',
                     'Species ':'Species',
                     'Fatal (Y/N)':'Fatal'}, 
                      inplace=True)

# Missing values

#### Las 24 columnas que hay son:

In [190]:
print(data.columns)

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species',
       'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')


**Vamos a estudiar el numero de valores nulos en esas columnas.**

In [191]:
# Mostramos columnas es las que es número de valores NaN es mayor que cero.
null_cols = data.isnull().sum()
null_cols[null_cols > 0]

Case Number                  1
Year                         2
Type                         4
Country                     50
Area                       455
Location                   540
Activity                   544
Name                       210
Sex                        565
Age                       2831
Injury                      28
Fatal                      539
Time                      3354
Species                   2838
Investigator or Source      17
href formula                 1
Unnamed: 22               6301
Unnamed: 23               6300
dtype: int64

**Como vemos, las columnas 'unnamed' no aportan nada en absoluto, son nulas enteras. Otras columnas donde hay muchos 
valores nulos es la de la hora del ataque, pero me parece importante dejar esa columna, más tarde la arreglaremos.
Columnas que vamos a eliminar:**
* Unnamed: no aportan nada.
* Year: información repetida en 'date'. A demás en date no falta ningún value.
* Case Number: no relevante para el estudio.
* Investigator of Source: no relevante para el estudio.
* Name: no relevante para el estudio.
* pdf: no relevante para el estudio.
* href:no relevante para el estudio.
* href formula: no relevante para el estudio.
* Case Number.1', 'Case Number.2': repetidas y no relevantes.
* 'original order': no relevante para el estudio.


In [192]:
drop_cols = ['Unnamed: 22', 'Unnamed: 23', 'Year', 'Case Number', 'Investigator or Source', 'Name', 
             'pdf', 'href','href formula', 'Case Number.1', 'Case Number.2', 'original order' ]
data = data.drop(drop_cols, axis=1)
data.head()

,Date,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal,Time,Species
0,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN
2,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN
3,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,NaN,Minor injury to lower leg,N,NaN,2 m shark
4,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m"


In [193]:
# Columnas restantes con número de valores NaN mayor que cero.
null_cols = data.isnull().sum()
null_cols[null_cols > 0]

Type           4
Country       50
Area         455
Location     540
Activity     544
Sex          565
Age         2831
Injury        28
Fatal        539
Time        3354
Species     2838
dtype: int64

**El número de datos NaN en age es similar al de especies, y el de location, activity y fatal también.¿Será por alguna razón?**

In [194]:
null_displ = data[(data['Species'].isnull()==True)]

* No encuentro relación entre la falta de datos en Fatal y otras columnas.
* Las filas en las que falta location coinciden con las que falta area, y no viceversa.
* Las filas en las que falta el sexo también falta la edad, y no viceversa.

**No he encontrado una relación muy significativa entre los datos**

# Fixing NaN values

In [195]:
null_displ = data[(data['Country'].isnull()==False)]
null_displ

,Date,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal,Time,Species
0,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN
2,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN
3,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,NaN,Minor injury to lower leg,N,NaN,2 m shark
4,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m"
...,...,...,...,...,...,...,...,...,...,...,...,...
6297,Before 1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,M,NaN,FATAL,Y,NaN,NaN
6298,Before 1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,M,NaN,FATAL,Y,NaN,NaN
6299,1900-1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,M,NaN,FATAL,Y,NaN,NaN
6300,1883-1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,M,NaN,FATAL,Y,NaN,NaN


### Aproximo a la moda

##### Type: aproximo a la media

In [196]:
data['Type'].mode()

0    Unprovoked
dtype: object

In [197]:
data.Type = data.Type.fillna('Unprovoked')

##### Species: aproximo a la media

In [198]:
data.Species.mode()

0    White shark
dtype: object

In [199]:
data.Species = data.Species.fillna('White shark')

##### Age: aproximo a la media

In [200]:
# Las edades son un string, tengo que ver una forma de hacerles la media

string_numbers = [str(i) for i in range(0,100)]

ages_str = [age for age in data.Age if age in string_numbers]

ages = [int(age) for age in ages_str]

sum(ages)/len(ages)

27.34582218257508

In [201]:
data.Age = data.Age.fillna(sum(ages)/len(ages))

##### Sex: aproximo a la moda

* Sex: Hay 310 mujeres frente  1835 hombres, por lo que sustituiremos los NaN por hombres

In [202]:
#data[data.Sex =='F'].value_counts()

In [203]:
data.Sex = data.Sex.fillna('M')

##### Injury: aproximo a la moda

In [204]:
data.Injury.mode()

0    FATAL
dtype: object

In [205]:
data.Injury = data.Injury.fillna('FATAL')

##### Fatal: aproximo a la moda

453 Letales frente a 2187 que no.

In [206]:
#data[data.Fatal=='N'].value_counts()

In [207]:
data.Fatal = data.Fatal.fillna('N')

##### Time: 

Faltan más de la mitad de los datos sobre la hora. Sin embargo, me sigue pareciendo interesante ver qué podemos obtener de los que tenemos. Los datos que hay son demasiado variados

In [211]:
data.Time.value_counts().head(50)

afternoon    1018
morning       636
evening       328
night         134
Name: Time, dtype: int64

(ya sé que esto es super cutre pero no lo he conseguido con regex)

In [209]:
morning_times   = ['06h00','06h30','07h00','07h30','08h00','08h30','09h00','09h30','10h00','10h30','11h00','11h30']
afternoon_times = ['12h00','12h30','13h00','13h30','14h00','14h30','15h00','15h30','16h00','16h30']
evening_times   = ['17h00','17h30','18h00','18h30','19h00','19h30']
night_times     = ['20h00','20h30','21h00','21h30','22h00','22h30','23h00','23h30','24h00','24h30','01h00','01h30',
                   '02h00','02h30','03h00','03h30','04h00','04h30','05h00','05h30']

def clean_time(x):
    if (x == 'Night') or (x in night_times):
        return 'night'
    
    elif (x =='Evening') or (x in evening_times):
        return 'evening'
    
    elif (x =='Morning') or (x in morning_times):
        return 'morning'
    
    elif (x =='Afternoon') or (x =='Late afternoon') or (x in afternoon_times):
        return 'afternoon'
    
    elif (x =='Dusk'):
        return 'evening'
    

In [210]:
data.Time = data.Time.apply(clean_time)
data.Time.value_counts().head(30)

afternoon    1018
morning       636
evening       328
night         134
Name: Time, dtype: int64

In [184]:
data.Time

0       evening
1          None
2          None
3          None
4          None
         ...   
6297       None
6298       None
6299       None
6300       None
6301       None
Name: Time, Length: 6302, dtype: object

##### Type: aproximo a la moda

##### Type: aproximo a la moda

##### Type: aproximo a la moda

##### Type: aproximo a la moda

In [33]:
# Columnas restantes con número de valores NaN mayor que cero.
null_cols = data.isnull().sum()
null_cols[null_cols > 0]

Country       50
Area         455
Location     540
Activity     544
Time        3354
dtype: int64

In [ ]:
data.Time = data.Time.to_

In [149]:
data.dtypes

Date        object
Type        object
Country     object
Area        object
Location    object
Activity    object
Sex         object
Age         object
Injury      object
Fatal       object
Time        object
Species     object
dtype: object